In [32]:
%matplotlib inline
from IPython.display import display, clear_output
import ipywidgets as widgets

In [29]:
%pip install --upgrade jupyter_contrib_nbextensions
!jupyter contrib nbextension install --user
!jupyter nbextension enable --py widgetsnbextension

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 8.2 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.9/466.9 KB 4.6 MB/s eta 0:00:005.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 5.4 MB/s eta 0:00:00m eta 0:00:010:00:01
Using legacy 'setup.py install' for jupyter_contrib_nbextensions, since package 'wheel' is not installed.
Using legacy 'setup.py install' for jupyter_contrib_core, since package 'wheel' is not installed.
  Running setup.py install for jupyter_contrib_core ... done
  Running setup.py install for jupyter_contrib_nbextensions ... done
Note: you may need to restart the kernel to use updated packages.
Traceback (most recent call last):
  File "/home/oem/PROJECTS/Sentiment_Analysis/sentiment_env/lib/python3.10/site-packages/jupyter_contrib_core/notebook_compat/nbextensions.py", line 6, in <module>
    from notebook.e

In [33]:
print("Widgets available:", widgets.Button is not None)

Widgets available: True


In [34]:
import pandas as pd
import numpy as np
import joblib
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# Load model and vectorizer
try:
    model =joblib.load('../models/best_sentiment_model_logistic_regression.pkl')
    tfidf = joblib.load('../models/tfidf_vectorizer.pkl')
    print("✅ Model and vectorizer loaded successfully!")
except:
    print("❌ Could not load model. Make sure you've run the training notebook first.")

✅ Model and vectorizer loaded successfully!


In [35]:
def preprocess_text(text):
    """Preprocess text for prediction"""
    # Convert to lowercase
    text = text.lower()

    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)

    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)

    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()

    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

# Test the preprocessing
test_text = "This movie was absolutely amazing! I loved every minute of it."
print(f"Original: {test_text}")
print(f"Processed: {preprocess_text(test_text)}")

Original: This movie was absolutely amazing! I loved every minute of it.
Processed: movie absolutely amazing loved every minute


In [40]:
def predict_sentiment(text):
    """Predict sentiment of given text"""
    if not text.strip():
        return "Please enter some text", 0.0, "neutral"
    # Preprocess text
    processed_text = preprocess_text(text)
    # Transform using TF-IDF
    text_tfidf = tfidf.transform([processed_text])
    # Predict
    prediction = model.predict(text_tfidf)[0]
    probabilities = model.predict_proba(text_tfidf)[0]
    confidence = max(probabilities)
    # Get probability for each class
    prob_negative = probabilities[0] if model.classes_[0] == 'negative' else probabilities[1]

In [27]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import random

# Mock prediction function for testing
def predict_sentiment(text):
    # Simulate analysis
    prob_pos = random.uniform(0, 1)
    prob_neg = 1 - prob_pos
    confidence = max(prob_pos, prob_neg)
    prediction = 'positive' if prob_pos > prob_neg else 'negative'
    return prediction, confidence, prob_neg, prob_pos

# Create interactive widgets
text_input = widgets.Textarea(
    value='Enter your text here...',
    placeholder='Type your review or text here',
    description='Text:',
    layout=widgets.Layout(width='100%', height='100px')
)

predict_button = widgets.Button(
    description='🔍 Analyze Sentiment',
    button_style='primary',
    layout=widgets.Layout(width='200px')
)

output_area = widgets.Output()

def on_predict_button_clicked(b):
    with output_area:
        clear_output()
        try:
            text = text_input.value
            if text and text.strip() and text != 'Enter your text here...':
                prediction, confidence, prob_neg, prob_pos = predict_sentiment(text)

                print("🎯 SENTIMENT ANALYSIS RESULTS")
                print("=" * 40)
                print(f"📝 Input Text: {text[:100]}{'...' if len(text) > 100 else ''}")
                print(f"🏷️ Predicted Sentiment: {prediction.upper()}")
                print(f"🎲 Confidence: {confidence:.2%}")
                print(f"📊 Probability Breakdown:")
                print(f"   😊 Positive: {prob_pos:.2%}")
                print(f"   😞 Negative: {prob_neg:.2%}")

                # Visualization
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

                ax1.barh(['Confidence'], [confidence], color=['green' if prediction == 'positive' else 'red'])
                ax1.set_xlim(0, 1)
                ax1.set_title('Prediction Confidence')
                ax1.set_xlabel('Confidence Score')

                sentiments = ['Negative', 'Positive']
                probabilities = [prob_neg, prob_pos]
                bars = ax2.bar(sentiments, probabilities, color=['lightcoral', 'lightgreen'])
                ax2.set_ylim(0, 1)
                ax2.set_title('Sentiment Probabilities')
                ax2.set_ylabel('Probability')

                for bar, prob in zip(bars, probabilities):
                    ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.01,
                             f'{prob:.2%}', ha='center', va='bottom')

                plt.tight_layout()
                plt.show()
            else:
                print("⚠️ Please enter some text to analyze!")
        except Exception as e:
            print(f"❌ Error: {str(e)}")

predict_button.on_click(on_predict_button_clicked)

# Display the UI
display(widgets.VBox([
    widgets.HTML("<h2> 🤖 Interactive Sentiment Analysis Tool</h2>"),
    text_input,
    predict_button,
    output_area
]))

In [17]:
# Sample texts for testing
sample_texts = [
    "This movie was absolutely fantastic! Great acting and storyline.",
    "Terrible film, complete waste of time and money.",
    "The movie was okay, nothing special but not bad either.",
    "I loved the cinematography but the plot was confusing.",
    "Worst movie I've ever seen, don't watch it!",
    "Amazing performances by all actors, highly recommended!",
    "The movie had its moments but overall disappointing.",
    "Brilliant direction and excellent screenplay.",
    "Boring and predictable, fell asleep halfway through.",
    "A masterpiece of modern cinema, truly inspiring!"
]
print("🧪 BATCH TESTING WITH SAMPLE TEXTS")
print("=" * 50)
results = []
for i, text in enumerate(sample_texts, 1):
    prediction, confidence, prob_neg, prob_pos = predict_sentiment(text)
    results.append({
        'Text': text[:50] + '...' if len(text) > 50 else text,
        'Prediction': prediction,
        'Confidence': f"{confidence:.2%}",
        'Positive_Prob': f"{prob_pos:.2%}",
        'Negative_Prob': f"{prob_neg:.2%}"
    })
emoji = "😊" if prediction == 'positive' else "😞"
print(f"{i:2d}. {emoji} {prediction.upper()} ({confidence:.2%}) - {text[:60]}{'...' if len(text) > 60 else ''}")
# Create results DataFrame
results_df = pd.DataFrame(results)
print(f"\n📊 DETAILED RESULTS:")
display(results_df)   

🧪 BATCH TESTING WITH SAMPLE TEXTS
10. 😊 POSITIVE (93.99%) - A masterpiece of modern cinema, truly inspiring!

📊 DETAILED RESULTS:


,Text,Prediction,Confidence,Positive_Prob,Negative_Prob
0,This movie was absolutely fantastic! Great act...,positive,94.69%,94.69%,5.31%
1,"Terrible film, complete waste of time and money.",negative,99.95%,0.05%,99.95%
2,"The movie was okay, nothing special but not ba...",negative,99.75%,0.25%,99.75%
3,I loved the cinematography but the plot was co...,positive,58.97%,58.97%,41.03%
4,"Worst movie I've ever seen, don't watch it!",negative,97.95%,2.05%,97.95%
5,"Amazing performances by all actors, highly rec...",positive,99.54%,99.54%,0.46%
6,The movie had its moments but overall disappoi...,negative,92.21%,7.79%,92.21%
7,Brilliant direction and excellent screenplay.,positive,97.60%,97.60%,2.40%
8,"Boring and predictable, fell asleep halfway th...",negative,99.27%,0.73%,99.27%
9,"A masterpiece of modern cinema, truly inspiring!",positive,93.99%,93.99%,6.01%
